In [ ]:
# @title Imports, initial setup (Ctrl+F9 to run all)
import os
import re
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
import copy

try:
    import gamry_parser
except:
    subprocess.run(["pip", "install", "gamry-parser"], encoding="utf-8", shell=False)
finally:
    import gamry_parser

gp = gamry_parser.CyclicVoltammetry()

print("Done.")

In [ ]:
"""
### SCRIPT CONFIGURATION SETTINGS ###
"""

# @markdown **Experimental Setup**

# @markdown Where should the notebook search for DTA files? Examples (using google colab):
# @markdown - Mounted google drive folder: `/content/drive/`
# @markdown - If uploading files manually, : `/content/`).

data_path = "/content/"  # @param {type:"string"}

# @markdown Filter which files we want to analyze
file_pattern = "Search-For-Text"  # @param {type:"string"}

# @markdown Extract trace labels from file name (e.g. `[17:].lower()` => drop the first 17 characters from the filename and convert to lowercase). The trace labels are used for category labeling (and plot legends)
file_label_xform = "[51:]"  # @param {type:"string"}

# create a "results" dataframe to contain the values we care about
data_df = pd.DataFrame()
settings_df = pd.DataFrame()
peaks_df = pd.DataFrame()

# identify files to process
files = [
    f
    for f in os.listdir(data_path)
    if os.path.splitext(f)[1].lower() == ".dta"
    and len(re.findall(file_pattern.upper(), f.upper())) > 0
]

In [ ]:
# @markdown **Process Data and Detect Peaks**

# @markdown Which CV curves (cycle number) should be sampled? (`0` would select the first CV curve from each file)
curves_to_sample = "0"  # @param {type:"string"}
curves_to_sample = [int(item.strip()) for item in curves_to_sample.split(",")]

# @markdown Peak Detection: specify the peak detection parameters
peak_width_mV = 75  # @param {type:"integer"}
peak_height_nA = 25  # @param {type:"integer"}
peak_thresh_max_mV = 800  # @param {type:"integer"}
peak_thresh_min_mV = -100  # @param  {type:"integer"}


# this method finds the row that has an index value closest to the desired time elapsed
def duration_lookup(df, elapsed):
    return df.index.get_loc(elapsed, method="nearest")


# iterate through each DTA file
for index, file in enumerate(files):
    print("Checking File {}".format(file))

    label, ext = os.path.splitext(file)
    my_label = "-".join(eval("label{}".format(file_label_xform)).strip().split())

    # load the dta file using gamry parser
    gp.load(filename=os.path.join(data_path, file))

    is_cv = gp.experiment_type == "CV"
    if not is_cv:
        # if the DTA file is a different experiment type, skip it and move to the next file.
        print("File `{}` is not a CV experiment. Skipping".format(file))
        del files[index]  # remove invalid file from list
        continue

    # for each CV file, let's extract the relevant information
    cv = gamry_parser.CyclicVoltammetry(filename=os.path.join(data_path, file))
    cv.load()
    for curve_num in curves_to_sample:
        print("\tProcessing Curve #{}".format(curve_num))
        v1, v2 = cv.v_range
        settings = pd.DataFrame(
            {
                "label": my_label,
                "curves": cv.curve_count,
                "v1_mV": v1 * 1000,
                "v2_mV": v2 * 1000,
                "rate_mV": cv.scan_rate,
            },
            index=[0],
        )
        settings_df = settings_df.append(settings)

        data = copy.deepcopy(cv.curve(curve=curve_num))
        data.Im = data.Im * 1e9
        data.Vf = data.Vf * 1e3
        data["label"] = my_label  # "{:03d}-{}".format(index, curve_num)

        data_df = data_df.append(data)

        # find peaks in the data
        dV = cv.scan_rate  # in mV
        peak_width = int(peak_width_mV / dV)
        peaks_pos, props_pos = find_peaks(
            data.Im, width=peak_width, distance=2 * peak_width, height=peak_height_nA
        )
        peaks_neg, props_neg = find_peaks(
            -data.Im, width=peak_width, distance=2 * peak_width, height=peak_height_nA
        )
        peaks = list(peaks_pos) + list(peaks_neg)
        # remove peaks that are out of min/max range
        peaks = [
            peak
            for peak in peaks
            if data.Vf.iloc[peak] >= peak_thresh_min_mV
            and data.Vf.iloc[peak] <= peak_thresh_max_mV
        ]

        # add detected peaks to aggregated peak dataframe
        peaks = data.iloc[peaks].sort_values(by="Vf")
        peaks["index"] = peaks.index
        peaks.reset_index(level=0, inplace=True)
        peaks_df = peaks_df.append(peaks)
        peaks_df = peaks_df[["label", "index", "Vf", "Im"]]
        # print("\tdetected peaks (mV)", [int(peak) for peak in data.iloc[peaks].Vf.sort_values().tolist()])

print("\nFile Metadata")
print(settings_df.to_string(index=False))

print("\nPeaks Detected")
print(peaks_df.to_string(index=False))

In [ ]:
# @markdown **I-V plot**: Overlay the loaded CyclicVoltammetry Curves

from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.colors import DEFAULT_PLOTLY_COLORS

fig = make_subplots(rows=1, cols=1, shared_xaxes=True, vertical_spacing=0.02)

for index, exp_id in enumerate(data_df.label.unique()):
    data = data_df.loc[data_df.label == exp_id]
    newTrace = go.Scatter(
        x=data.Vf,
        y=data.Im,
        mode="lines",
        name=exp_id,
        legendgroup=files[index],
        line=dict(color=DEFAULT_PLOTLY_COLORS[index]),
    )
    fig.add_trace(newTrace, row=1, col=1)
    peak = peaks_df.loc[peaks_df.label == exp_id]
    newTrace = go.Scatter(
        x=peak.Vf,
        y=peak.Im,
        mode="markers",
        showlegend=False,
        marker=dict(
            size=12,
            color=DEFAULT_PLOTLY_COLORS[index],
        ),
    )
    fig.add_trace(newTrace, row=1, col=1)

layout = {
    "title": {
        "text": "Cyclic Voltammetry Overlay",
        "yanchor": "top",
        "y": 0.95,
        "x": 0.5,
    },
    "xaxis": {"anchor": "x", "title": "voltage, mV"},
    "yaxis": {"title": "current, nA", "type": "linear" ""},
    "width": 1200,
    "height": 500,
    "margin": dict(l=30, r=20, t=60, b=20),
}
fig.update_layout(layout)

config = {
    "displaylogo": False,
    "modeBarButtonsToRemove": [
        "select2d",
        "lasso2d",
        "hoverClosestCartesian",
        "toggleSpikelines",
        "hoverCompareCartesian",
    ],
}
fig.show(config=config)